In [6]:
# Import selenium and its dependencies
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

# import pandas 
import pandas as pd
import time
base_url = "https://members.ocpinfo.com/tcpr/public/pr/en/#/forms/new/?table=0x800000000000003D&form=0x800000000000002C&command=0x80000000000007C5&donotreload=ShowSRL"

import undetected_chromedriver as uc
driver = uc.Chrome(use_subprocess=False)

scrape_df = pd.DataFrame()
scrape_df["name"] = ""
scrape_df["href"] = ""

In [18]:
scrape_df = pd.DataFrame()
scrape_df["name"] = ""
scrape_df["href"] = ""
for i in range(1, 49):
    
    print(f"Page: {i}")
    
    # go the page url and wait 2 seconds for the page to load
    driver.get(f"{base_url}{i}")
    time.sleep(5)
    
    # check if the element is present
    try:
        element_present = EC.presence_of_element_located((By.ID, 'SearchReasultList'))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    
    listings = driver.find_elements(By.CLASS_NAME, "ResultDiv")
    
    for l in listings:
        try:
            name = l.text
            href = l.find_element(By.TAG_NAME, "a").get_attribute("href")
            data = {
                "name": name,
                "href": href,
            }
            scrape_df = pd.concat([scrape_df, pd.DataFrame([data])], ignore_index=True)
            print(f"name: {name}")
        except NoSuchElementException:
            continue

Page: 1
name: Aarav Shah
Pharmacist
Biostan Pharmacy
Orangeville, ON
Can provide patient care
name: Aaron Nixon
Pharmacist
Main Street Pharmacy and Wellness Centre
Kingsville, ON
Can provide patient care
name: Abeer (Abby) Elia
Pharmacist
LimeGarth Pharmacy
Hamilton, ON
Can provide patient care
name: Abhay Patel
Pharmacist
Town Square Pharmacy
New Hamburg, ON
Can provide patient care
name: Adam Silvertown
Pharmacist
Pace Pharmacy
East York, ON
Can provide patient care
name: Adebola Banjoko
Pharmacist
The K-W Pharmacy
Waterloo, ON
Can provide patient care
name: Adel Abdelmalek
Pharmacist
Main Drug Mart
Scarborough, ON
Can provide patient care
name: Adel Tadros
Pharmacist
F C Friendly Care Baythorn Pharmacy
Thornhill, ON
Can provide patient care
name: Adesh (adesh) Vora
Pharmacist
Does not provide patient care
name: Ahmed Abou Zeid
Pharmacist
Bracebridge Compounding Pharmacy
Bracebridge, ON
Can provide patient care
Page: 2
name: Ahmed Hassan
Pharmacist
Model Guardian Drug Store Limited
C

In [30]:
for iter, row in scrape_df[scrape_df['graduation year'].isna()].iterrows():
    if row["graduation year"] is None:
        print(f"skipping {row['name']}")
        continue
    driver.get(row["href"])
    time.sleep(3)
    driver.find_element(By.ID, "pid7").click()
    time.sleep(1)
    scrape_df.loc[iter, "graduation year"] = driver.find_element(By.ID, "qualiEducationResult").find_elements(By.CLASS_NAME, "f16")[1].text


In [32]:
scrape_df['graduation year'] = scrape_df['graduation year'].str[-4:]

In [38]:
scrape_df['age'] = (2022 - scrape_df['graduation year'].astype(int)) + 25
scrape_df.to_csv("ocp_retiring_soon_pharmacist.csv", index=False)